This notebook allows for generating articles and summaries, along with the gold summary.

In [1]:
import rinokeras
from rinokeras.models.transformer import Transformer
import tensorflow.contrib.eager as tfe
from Dataset import Dataset
import tensorflow as tf
import numpy as np
import tqdm
import os, time
import matplotlib.pyplot as plt

Replace the json below with the newest dataset. 

In [3]:
dataset = Dataset("nl_summaries_small_eval.0.4.json")
dataset.load()

input_length = 400; output_length = 100; dim = 304
%load_ext autoreload
%autoreload 2
offset = 3 # Hack for now, due to beginning of word vocab
padI = dataset.vocab.index('<PAD>')
n_vocab = len(dataset.vocab)
print(n_vocab)
print(padI)

32003
1


# Eager mode

In [4]:
tf.enable_eager_execution()

In [5]:
model = Transformer(discrete=True, n_layers=3, n_symbols_in=n_vocab, d_model=dim, n_symbols_out=n_vocab, embedding_initializer=np.zeros((n_vocab, dim)), share_source_target_embedding=True)

# Very hacky: do a forward run to force the model to be built, before loading the weights.
encode, decode, mask, decode_mask = dataset.build_batch(['input', 'output', 'input_mask', 'output_mask'], size=64, cut='evaluation', types=[tf.int32,tf.int32,tf.bool,tf.bool])
decoded_logits = model(encode, decode, encoder_mask=mask, decoder_mask=decode_mask)

Replace the weight file below with the latest weight file. 

In [6]:
model.load_weights("summarize_adam_ckpt.h5")

In [7]:
# Testing loss in eager mode

encode, decode, mask, decode_mask = dataset.build_batch(['input', 'output', 'input_mask', 'output_mask'], size=64, cut='evaluation', types=[tf.int32,tf.int32,tf.bool,tf.bool])

decoded_logits = model(encode, decode, encoder_mask=mask, decoder_mask=decode_mask)
test_loss = tf.losses.sparse_softmax_cross_entropy(decode, decoded_logits, tf.cast(decode_mask, tf.float32))

# test_loss = tf.contrib.seq2seq.sequence_loss(decoded_logits, decode, tf.cast(decode_mask, tf.float32))
print("Eval loss:",float(test_loss))

Eval loss: 2.636415481567383


In [9]:
import os
#Removes existing weight numpy files to avoid confusion. 
def remove_weights():
    if os.path.exists("data/input_self_attention.npy"):
        os.remove("data/input_self_attention.npy")
    if os.path.exists("data/cross_attention.npy"):
        os.remove("data/cross_attention.npy")
    if os.path.exists("data/output_self_attention.npy"):
        os.remove("data/output_self_attention.npy")

In [10]:
def reshape_numpy_data():
    if os.path.exists("data/cross_attention.npy"):
        input_data = np.load("data/cross_attention.npy").reshape((6, 8, 100, 400))
        np.save("data/cross_attention.npy", input_data)

In [8]:
def trim_data(dataset, encode, fstyle):
    input_list = list((dataset.vocab[i]) for i in np.array(encode[0]))
    output_list = list((dataset.vocab[i]) for i in np.array(fstyle_words[0]))
    val_1 = input_list.index('</s>') if '</s>' in input_list else 400
    val_2 = output_list.index('</s>') if '</s>' in output_list else 100
    
    cross = np.load('data/cross_attention.npy')
    cross = cross[:, :, :val_2, :val_1]
    np.save('data/cross_attention.npy', cross)
    
    isa = np.load('data/input_self_attention.npy')
    isa = isa[:, :, :val_1, :val_1]
    np.save('data/input_self_attention.npy', isa)

In [14]:
for i in tqdm.tqdm_notebook(range(1)):
    remove_weights()
    encode, decode, encoder_mask, decode_mask = dataset.build_batch(['input', 'output', 'input_mask', 'output_mask'], size=1, cut='evaluation', types=[tf.int32,tf.int32,tf.bool,tf.bool])

    fstyle_decode_logits = model.test_decode(encode, output_length, encoder_mask=encoder_mask) # 
    fstyle_words = fstyle_decode_logits.numpy() + offset

    decode += offset #Hack
    encode += offset #Hack
    
    article_content = dataset.evaluate_sentence(encode.numpy())[0]
    real_summary = dataset.evaluate_sentence(np.array(decode))[0]
    
    fstyle_summ = dataset.evaluate_sentence(fstyle_words)[0]
    #tstyle_summ = dataset.evaluate_sentence(tstyle_words)[0]
    
    reshape_numpy_data()
    trim_data(dataset, encode, fstyle_words)
    #print ("This sentence loss:",test_loss.numpy())
    print("News article:")
    print(article_content)
    print("---------------")
    print("Gold summary:",real_summary)
    print("---------------")
    print("Fast decode summary:",fstyle_summ)
    print("---------------")

News article:
Ministry of Defence has withdrawn its attempt to strike out a whistleblowing claim brought by a doctor who raised concerns about alleged discrepancies in the dispensing of strong painkillers at an army base where he was working. Dr Stephen Frost, a civilian doctor who had worked with the military for 20 years, was dismissed three years ago by text and email while on a family holiday, and since then has sought to find out why and for redress from the MoD. At a hearing in Manchester, the MoD confirmed it had withdrawn its application to have Frost's claims struck out. John Hendy C, for Frost, told the tribunal the parties had reached a "degree of agreement". The hearing was adjourned so that discussions about a possible settlement could take place. No details of why the MoD had changed its approach were given. Frost, who was involved in the campaign for a full inquest on the weapons inspector David Kelly, who died at the height of the Iraq dossier scandal, described the las